# COSMO Project
By Mathilde Raynal, Etienne Bonvin and Xavier Pantet

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import numpy as np
from regressions import *
import pandas as pd

In [3]:
DATA_FOLDER = "data/"
#X = np.load(DATA_FOLDER + "feature_mat_radial_compression.npy")
X = np.load(DATA_FOLDER + "pca_x.npy")
y = np.load(DATA_FOLDER + "CSD500-r_train-H_total.npy")

In [4]:
print("X: " + str(X.shape))
print("y: " + str(y.shape))

X: (30049, 3004)
y: (30049,)


## Idea 1: Good ol' least squares

We first try a standard and naive implementation of `least_squares`:

In [5]:
w_star = least_squares(y, X)
loss = rmse(y, X, w_star)
print("Loss = " + str(loss))

Loss = 26.166553759575127


We see that the loss is quite large! We hope to do better using polynomial expansion. We use 4-fold cross-validation to find the best `degree`:

In [21]:
k_fold = 4
k_indices = build_k_indices(y, k_fold)

rmse_tr = []
rmse_te = []

for degree in range(4):
    rmse_tr_tmp = []
    rmse_te_tmp = []
    for k in range(k_fold):
        loss_tr, loss_te, _ = cross_validation(y, X, k_indices, k, degree, least_squares)
        rmse_tr_tmp.append(loss_tr)
        rmse_te_tmp.append(loss_te)
    rmse_tr.append(np.mean(rmse_tr_tmp))
    rmse_te.append(np.mean(rmse_te_tmp))

In [22]:
pd.DataFrame([rmse_tr, rmse_te]).add_prefix("Degree ").rename({0: "Train error", 1: "Test error"})

,Degree 0,Degree 1,Degree 2,Degree 3
Train error,3.214364,0.595601,0.466134,0.356275
Test error,3.214384,0.710750,0.748990,1.042313


Indeed, polynomial expansion provides better results. Moreover, we see that the best `degree` is 2.